<img src="Figs/GEOS_logo.pdf" width="500"/>

# Sample InSAR data: <font color=blue>"change_InSAR_ref.ipynb"</font>
#### Nov 16, 2021  <font color=red>(v. testing)</font> 
##### Jeonghyeop Kim (jeonghyeop.kim@gmail.com)

> input files : **`DT42_data_final.dat`** & **`AT35_data_final.dat`** & **`ref_GNSS.dat`**\
> output files: **`DT42_data_new_ref.dat`** & **`AT35_data_new_ref.dat`**

0. This code is a part of the joint inversion project (project4: joint inversion of GNSS and InSAR)
1. This code changes the frame of reference to a point. 
2. The LOS displacement at the ref. point will be projected onto each of the pointing vectors for all the data points. 
3. The projected LOS displacements will be subtracted from the original data
4. DUE TO THE NATURE OF INSAR measurement, maybe to obtain basis-function responses with respect to InSAR reference would be better approach.
5. To avoid to pick an outlier, this code finds **`30`** closest InSAR data points to the given ref. point, and then uses the median value as the reference LOS displacement.
6. The given ref point is the reference of the GNSS data : `ref_GNSS.dat` 

> [**unwrapping**] (The LOS deformations are relative motions to a point and the point should be close to the signal). \
> **For now (10/04/2021)**, a master code for the boundary condition velocity responses is NOT ready. \
> Thus, this alternative method is used. \
> In addition, use of different sampling approaches for InSAR data (e.g., QuadTree) may not guarantee that \
> a data is available at the top right corner (the reference of the basis-function responses). 


In [1]:
#import
import numpy as np
import pandas as pd
from distance_calculation import distance_calculation

In [2]:
# Define reference point
ref_file="ref_GNSS.dat"
df_ref = pd.read_csv(ref_file, header=None, sep = ' ')
lon_ref = df_ref.iloc[0,0]
lat_ref = df_ref.iloc[0,1]

In [3]:
# Load InSAR data (in InSAR reference)

names = ['lon','lat','rate','Px','Py','Pz']

input_file_1="DT42_data_final.dat"
df_DESD=pd.read_csv(input_file_1, header=None, sep = ' ')
df_DESD.columns = names

input_file_2="AT35_data_final.dat"
df_ASD=pd.read_csv(input_file_2, header=None, sep = ' ')
df_ASD.columns = names


In [4]:
NUM = len(df_DESD)
dist_array = np.zeros((NUM,))

for i in range(NUM):
    lat_df = df_DESD.iloc[i,1]
    lon_df = df_DESD.iloc[i,0]
    dist,az = distance_calculation(lat_ref,lon_ref,lat_df,lon_df)
    dist_array[i,] = dist
idx_closest=dist_array.argsort()[:30]

In [5]:
df_ASD_closest=df_ASD.iloc[idx_closest,:]
rate_ASD_ref=df_ASD_closest['rate'].median()
px_ASD_ref=df_ASD_closest['Px'].mean()
py_ASD_ref=df_ASD_closest['Py'].mean()
pz_ASD_ref=df_ASD_closest['Pz'].mean()

In [6]:
df_DESD_closest=df_DESD.iloc[idx_closest,:]
rate_DESD_ref=df_DESD_closest['rate'].median()
px_DESD_ref=df_DESD_closest['Px'].mean()
py_DESD_ref=df_DESD_closest['Py'].mean()
pz_DESD_ref=df_DESD_closest['Pz'].mean()

In [7]:
# LOS displacements - Projected ref. LOS displacements = rate - rate_ref * (Px*Px_ref + Py*Py_ref + Pz*Pz_ref)
df_ASD['new_rate']=df_ASD['rate']-rate_ASD_ref*(df_ASD['Px']*px_ASD_ref + df_ASD['Py']*py_ASD_ref + df_ASD['Pz']*pz_ASD_ref)
df_DESD['new_rate']=df_DESD['rate']-rate_DESD_ref*(df_DESD['Px']*px_DESD_ref + df_DESD['Py']*py_DESD_ref + df_DESD['Pz']*pz_DESD_ref)

In [8]:
cols_to_save = ['lon','lat','new_rate','Px','Py','Pz']
df_ASD_save = df_ASD.loc[:, cols_to_save]
df_DESD_save = df_DESD.loc[:, cols_to_save]

In [9]:
# Save data
df_ASD_save.to_csv('AT35_data_new_ref.dat',index=None, header=None, sep=' ',float_format='%.5f')
df_DESD_save.to_csv('DT42_data_new_ref.dat',index=None, header=None, sep=' ',float_format='%.5f')